In [1]:
import numpy as np
import tensorflow as tf

from pureEM import *

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


/Users/abeukers/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# PM task ideas

- network given the PM cue on first trial, must respond with PM action
    - on first trial encode stim_rep(PMcue):cell_state
    - retrieval gate controled by sensory info and cell
    - PM cue presented unifrom random after trial 25
    - trainable initial cell state
    - episode unique (random) PM cue
- goal of this first test is to see if retrieval gate is learnable
    - can network shut off retrieval during ongoing task?
- since during training I am giving the network feedback on every stimulus, how do I prevent it from just learning a mapping from stimulus to output in cell state? 
- maybe the first thing to check is pureWM model error rate on PM trials increase with OG task difficulty. then can I mitigate this with an EM architecture?

- 2back and 3back in same episode on random vectors 
    - DND has two slots corresponding to 2back and 3back
    - encoding happens when task shifts
    - learn to retrieve relevant cell state when task shifts

In [2]:
net = PMNet(40,nback=2,num_og_tokens=3,seed=2)
trainer = Trainer(net)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.


In [18]:
task = NBackPMTask(2,3)
X,Y = task.gen_seq(30)
Y.shape

(1, 30)

In [4]:
for i in range(1,1):
  print(i)

In [3]:
train_acc = trainer.train_loop(10000,100)

0.0 0.575
0.05 0.725
0.1 0.325
0.15 0.6
0.2 0.4
0.25 0.4
0.3 0.5
0.35 0.15
0.4 0.2
0.45 0.65


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(14,4))
plt.plot(train_acc)
for t in range(0,10000,100):
  plt.axvline(t,c='r',lw=.2)
plt.ylabel('accuracy')
plt.xlabel('training episode/epoch')
plt.title('LSTM20 training accuracy')
# plt.savefig('figures/nback+pm/training_acc-2back_2tokens')

In [ ]:
plt.figure(figsize=(13,4))
eps = 100
eval_acc = trainer.eval_loop(eps,25)
M = eval_acc.mean(0)
S = eval_acc.std(0)/np.sqrt(eps)
plt.plot(M,c='blue')
plt.fill_between(range(len(M)),M-S,M+S,alpha=.3)
plt.scatter(range(len(M)),M,c='blue')
for t in [10,15]:
  plt.axvline(t,c='g',lw=1)
plt.ylim(0,1.05)
plt.ylabel('mean acc')
plt.xlabel('trial')
plt.title('LSTM20 eval accuracy')
# plt.savefig('figures/nback+pm/eval_acc-2back_2tokens')